<h1 style="text-align : center"> FIXED POINT PATCH CODE REVIEW </h1>

# Table of Contents
* [Introduction](#Introduction)
* [Glossary](#Glossary)
* [Review](#Review)
	* [/include/clang/Basic/TokenKinds.def](#/include/clang/Basic/TokenKinds.def)
	* [/include/clang/AST/BuiltinTypes.def](#/include/clang/AST/BuiltinTypes.def)
	* [/include/clang/AST/Type.h](#/include/clang/AST/Type.h)
	* [/lib/AST/Type.cpp](#/lib/AST/Type.cpp)
	* [/lib/CodeGen/CGBuiltin.cpp](#/lib/CodeGen/CGBuiltin.cpp)
	* [/include/clang/Basic/Specifiers.h](#/include/clang/Basic/Specifiers.h)
	* [/include/clang/Basic/TargetInfo.h](#/include/clang/Basic/TargetInfo.h)
	* [/lib/Basic/TargetInfo.cpp](#/lib/Basic/TargetInfo.cpp)
	* [/include/clang/Sema/DeclSpec.h](#/include/clang/Sema/DeclSpec.h)
	* [/lib/Parse/ParseDecl.cpp - ParseExpr.cpp - ParseExprCxx.cpp - ParseTentative.cpp](#/lib/Parse/ParseDecl.cpp---ParseExpr.cpp---ParseExprCxx.cpp---ParseTentative.cpp)
	* [/include/clang/Lex/LiteralSupport.h](#/include/clang/Lex/LiteralSupport.h)
	* [/lib/Lex/LiteralSupport.cpp](#/lib/Lex/LiteralSupport.cpp)
	* [/include/clang/AST/ASTContext.h](#/include/clang/AST/ASTContext.h)
	* [/lib/AST/ASTContext.cpp](#/lib/AST/ASTContext.cpp)
	* [/lib/AST/ExprConstant.cpp](#/lib/AST/ExprConstant.cpp)
	* [/lib/CodeGen/CGExprConstant.cpp](#/lib/CodeGen/CGExprConstant.cpp)
	* [/lib/CodeGen/CGExprScalar.cpp](#/lib/CodeGen/CGExprScalar.cpp)
	* [/include/clang/AST/Expr.cpp](#/include/clang/AST/Expr.cpp)
	* [/lib/Sema/SemaExpr.cpp](#/lib/Sema/SemaExpr.cpp)
	* [/lib/Sema/SemaOverload.cpp](#/lib/Sema/SemaOverload.cpp)
	* [/lib/Sema/SemaType.cpp](#/lib/Sema/SemaType.cpp)
	* [/lib/Serialization/ASTCommon.cpp - ASTReader.cpp](#/lib/Serialization/ASTCommon.cpp---ASTReader.cpp)
* [Questions](#Questions)


# Introduction

1. This patch includes the diffs for the lexer, parser, sema, and conversion to bitcode (clang/lib/CodeGen).
2. The diffs are against the **LLVM 3.7**.
3. The patch **doesn’t support unsigned nor unsaturated _Fract or _Accum types**

# Glossary

- [**Type**](http://clang.llvm.org/doxygen/classclang_1_1Type.html) : The base class of the type hierarchy.
- [**QualType**](http://clang.llvm.org/doxygen/classclang_1_1QualType.html) : Wrapper class that has two pointers as attributes: one points to the Type and the other points to the Qualifier (Both of these classes are defined in Type.h). This greatly reduces the number of nodes we need to allocate for types. For the sake of semplicity we will consider QualType equals to CanQualType
- [**BuiltinType**](http://clang.llvm.org/doxygen/classclang_1_1BuiltinType.html#details) : Builtin types are always canonical and have a literal name field.

<img  src="FQWHQFGMBF9OHV9MVBMGARA9R4UYDU95.png"/>

- [**TST**](http://clang.llvm.org/doxygen/namespaceclang.html#a033691a5f00979c1a22d8aa114d07e13) : The Type Specifier Type (TST) is used to just specifies the type using an enum.
- [**DeclSpec**](http://clang.llvm.org/doxygen/classclang_1_1DeclSpec.html) : A Declaration Specifier encompasses storage-class-specifiers, type-specifiers, type-qualifiers, and function-specifiers. It basically describes the content of a declaration.
- [**Sema**](http://clang.llvm.org/doxygen/classclang_1_1Sema.html): Implements the semantic analysis and AST building for C.
- **CG_something.cpp** : Every class that begins with **CG** implements the code that will emits llvm intermediate represetation from AST nodes.

# Review

## /include/clang/Basic/TokenKinds.def

In this file has been added the definition of the token that the lexer will find inside the source code:
```C++
KEYWORD(__int96    , KEYALL)
KEYWORD(_Sat         , KEYALL)   // As specified by the ISO/IEC TR 18037
KEYWORD(_Fract      , KEYALL)   // As specified by the ISO/IEC TR 18037
KEYWORD(_Accum   , KEYALL)   // As specified by the ISO/IEC TR 18037
```

## /include/clang/AST/BuiltinTypes.def

In this file is defined the macro that will creates the IDs for the new added types. These IDs are fundamental in order to read and write correctly the AST.
```C++
+ #ifndef FIXED_TYPE
+ #define FIXED_TYPE(Id, SingletonId) BUILTIN_TYPE(Id, SingletonId)
+ #endif

+ FIXED_TYPE(SatSignedShortFract, FractTy)
+ FIXED_TYPE(SatSignedFract, FractTy)
+ FIXED_TYPE(SatSignedLongFract, FractTy)
+ FIXED_TYPE(SatSignedLongLongFract, FractTy)
+ 
+ FIXED_TYPE(SatSignedShortAccum, AccumTy)
+ FIXED_TYPE(SatSignedAccum, AccumTy)
+ FIXED_TYPE(SatSignedLongAccum, AccumTy)
+ FIXED_TYPE(SatSignedLongLongAccum, AccumTy)
```

## /include/clang/AST/Type.h

This file is used to represents types for languages in the C family. Here are specified all the features of the new types and are defined all the methods in order to access and create them.
```C++
+   struct FixedPointTypeTraits : public BuiltinTypeBitfields {
+     unsigned char GuardBits;
+     unsigned char Width;
+   };

+   bool isSaturated() const;
+   bool hasSameFixedPointRepresentation(const Type*) const;
+   bool isFractType() const;
+   bool isAccumType() const;
+   bool isFractOrAccumType() const;
+   bool isRealOrFixed() const;
+   bool isSignedFixedPoint() const;
+   bool isUnsignedFixedPoint() const;
+   unsigned getFixedPointGuardBits() const;
+   unsigned getFixedPointWidth() const;

+   void initFixedPointType(unsigned guardBits, unsigned width) {
+     FixedPointTraits.GuardBits = guardBits;
+     FixedPointTraits.Width = width;
+   }
+   const FixedPointTypeTraits& getFixedPointTypeTraits() const { return  FixedPointTraits; }
```

These attributes, like the guard bits, may be useful in the accumulation to prevent overflow.

## /lib/AST/Type.cpp

Implemented the functions defined in Type.h such as:

- **isFractType**

This snippet of code simply checks if the id of the type is included in the bounds that identifies the fract type

```C++
+ bool Type::isFractType() const {
+   if (const BuiltinType *BT = dyn_cast<BuiltinType>(CanonicalType))
+     return BT->getKind() >= BuiltinType::SatSignedShortFract &&
+       BT->getKind() <= BuiltinType::SatSignedLongLongFract;
+   return false;
+ }
```

- **isRealOrFixed()**

returns true if the the Type is real or fixed

```C++
+ bool Type::isRealOrFixed() const {
+   return isRealFloatingType() || isFractOrAccumType();
+ }
```

- **isFractOrAccumType()**

Used in order to obtain a value for the node FixedPointLiteral during the building of the AST.

```C++
+ bool Type::isFractOrAccumType() const {
+   if (const BuiltinType *BT = dyn_cast<BuiltinType>(CanonicalType))
+     return BT->getKind() >= BuiltinType::SatSignedShortFract &&
+       BT->getKind() <= BuiltinType::SatSignedLongLongAccum;
+   return false;
+ }
```

## /lib/CodeGen/CGBuiltin.cpp

Added inside the function **EmitBuiltinExpr** the code that will return a ConstantInteger that will represent the fixed point value in the llvm intermediate representation.

**NB :** The fixed point type are managed like int type by the llvm!
```C++
+     const Type* DestType = E->getType()->getUnqualifiedDesugaredType();
+     if (Result.Val.isFloat() && DestType->isFractOrAccumType()) {
+ 
+       APFloat F = Result.Val.getFloat();
+ 
+       unsigned Width = DestType->getFixedPointWidth();
+       llvm::APSInt fixedPointBits(Width, /*isUnsigned=>*/ !F.isNegative());
+       llvm::FixedPointFormat Format(DestType->isSignedFixedPoint(), DestType->getFixedPointGuardBits(), Width);
+ 
+       F.getMachineCodeValue(fixedPointBits, llvm::APFloat::rmTowardZero, Format);
+       unsigned Size = getContext().getTypeSize(DestType);
+       if (Size > Width) {
+ 	fixedPointBits = fixedPointBits.extend(Size);
+       }
+       ConstantInt* CI = llvm::ConstantInt::get(getLLVMContext(), fixedPointBits);
+ 
+       return RValue::get(CI);
+     }
```

## /include/clang/Basic/Specifiers.h

This file defines various enumerations that describe declarations and type specifiers. It has been added in these enumeration the cases for the new types.
```C++
+     TST_fract,
+     TST_accum,
+     TST_int96,
```

## /include/clang/Basic/TargetInfo.h

this file exposes all the features of the target architecture for which the code will be generated, and here are defined the method that return the characteristics of the new types according to the architecture informations.
```C++
+   unsigned getShortFractWidth() const { return ShortFractWidth; }
+   unsigned getShortFractAlign() const { return ShortFractAlign; }
+   unsigned getFractWidth() const { return FractWidth; }
+   unsigned getFractAlign() const { return FractAlign; }
+   unsigned getLongFractWidth() const { return LongFractWidth; }
+   unsigned getLongFractAlign() const { return LongFractAlign; }
+   unsigned getLongLongFractWidth() const { return LongLongFractWidth; }
+   unsigned getLongLongFractAlign() const { return LongLongFractAlign; }
+   unsigned getShortAccumWidth() const { return ShortAccumWidth; }
+   unsigned getShortAccumAlign() const { return ShortAccumAlign; }
+   unsigned getAccumWidth() const { return AccumWidth; }
+   unsigned getAccumAlign() const { return AccumAlign; }
+   unsigned getLongAccumWidth() const { return LongAccumWidth; }
+   unsigned getLongAccumAlign() const { return LongAccumAlign; }
+   unsigned getLongLongAccumWidth() const { return LongLongAccumWidth; }
+   unsigned getLongLongAccumAlign() const { return LongLongAccumAlign; }
+   virtual unsigned getNumGuardBits() const { return 8; }
+   virtual unsigned getShortAccumGuardBits() const { return 8; }
+   virtual unsigned getAccumGuardBits() const { return 8; }
+   virtual unsigned getLongAccumGuardBits() const { return 8; }
+   virtual unsigned getLongLongAccumGuardBits() const { return 8; }
```

## /lib/Basic/TargetInfo.cpp

In this file are defined all the characteristics of the new types according to the specific
```C++
+   ShortFractWidth = 8;
+   ShortFractAlign = 8;
+   FractWidth = 16;
+   FractAlign = 16;
+   LongFractWidth = 16;
+   LongFractAlign = 16;
+   LongLongFractWidth = 32;
+   LongLongFractAlign = 32;
+   //
+   // NOTE: sizeof(accum) will be rounded up using AccumAlign
+   //
+   ShortAccumWidth = 32+8;	// 8-bit * 8-bit accumulator with 8 guard bits
+   ShortAccumAlign = 32;
+   AccumWidth = 32+8;		// 16-bit * 16-bit accumulator with 8 guard bits
+   AccumAlign = 32;
+   LongAccumWidth = 32+8;	// 16-bit * 16-bit accumulator with 8 guard bits
+   LongAccumAlign = 32;
+   LongLongAccumWidth = 64+8;	// 32-bit * 32-bit accumulator with 8 guard bits
+   LongLongAccumAlign = 32;
```

## /include/clang/Sema/DeclSpec.h

Added the new Type Specifier Type for the new type. 
```C++
static const TST TST_fract = clang::TST_fract;
static const TST TST_accum = clang::TST_accum;
static const TST TST_int96 = clang::TST_int96;
```

Added the function **setSaturated** in order to memorize this information while parsing the declaration
The function setSaturated is used inside **Parser::ParseDeclarationSpecifiers** when meet a kw__Sat.

Saturated is a global variable inside this file that indicates if a declaration is a Sat or not
```C++
void setSaturated(SourceLocation l) { Saturated = true; SaturatedLoc = l; }
bool isSaturated() const { return Saturated; }
```

## /lib/Parse/ParseDecl.cpp - ParseExpr.cpp - ParseExprCxx.cpp - ParseTentative.cpp

in the function **Parser::ParseDeclarationSpecifiers** added the cases in order to handle the new type. For example in the case of the kw__Sat we are going to mark as Saturated the DS, using the function previously defined setSaturated().
```C++
+     case tok::kw__Sat:
+       if (DS.isSaturated()) Diag(Tok, diag::err_sat_specified);
+       DS.setSaturated(Tok.getLocation());
+       break;
+     case tok::kw__Fract:
+       isInvalid = DS.SetTypeSpecType(DeclSpec::TST_fract, Loc, PrevSpec, DiagID, Policy);
+       break;
+     case tok::kw__Accum:
+       isInvalid = DS.SetTypeSpecType(DeclSpec::TST_accum, Loc, PrevSpec, DiagID, Policy);
+       break;
+     case tok::kw___int96:
+       isInvalid = DS.SetTypeSpecType(DeclSpec::TST_int96, Loc, PrevSpec, DiagID, Policy);
+       break;
+ 
```

Added in the function **Parser::isKnownToBeTypeSpecifier** the cases in order to understand if we have a known type specifier.
```C++
+   case tok::kw__Sat:
+   case tok::kw__Fract:
+   case tok::kw__Accum:
+   case tok::kw___int96:
```

## /include/clang/Lex/LiteralSupport.h

This file defines NumericLiteralParser, CharLiteralParser and StringLiteralParser interfaces.
- **NumericLiteralParser** : This performs strict semantic analysis of the content of a ppnumber (a preprocessor number that is a sequence of digits and other characters which make up an integer or floating literal), classifying it as either integer, floating, or erroneous, determines the radix of the value and can convert it to a useful value.
- **CharLiteralParser** : Perform interpretation and semantic analysis of a character literal.
- **StringLiteralParser** : This decodes string escape characters and perform wide string analysis and perform concatenation of string


Added, in the NUmericLiteralParser class, the boolean attributes and their getter in order to handle also the property of the new types
```C++
+   bool isHalf;		// 'h'
+   bool isFract;		// 'r'
+   bool isAccum;		// 'k'

+   bool isFixedPointLiteral() const {
+     return isFloatingLiteral() && (isFract || isAccum);
+   }

+   llvm::APFloat::opStatus GetFixedPointValue(llvm::APFloat& Result);
```

## /lib/Lex/LiteralSupport.cpp

Implemented the cases in order to tag the parsed fixed point values using the methods and attributes defined in the header.

For example :

```C++
+     case 'h':
+     case 'H':
+       if (isHalf) break;		// Cannot be repeated.
+       if (isFloat || isImaginary) break;// Invalid
+       if (isLongLong) break;		// Invalid
+       isHalf = true;
+       continue;  // Success.
```

- **NumericLiteralParser::GetFixedPointValue(llvm::APFloat &Result)**

It checks if the type identifier is included in the bounds reserved to the fixed point types (specified in the class "BuiltinType")

```C++
+ llvm::APFloat::opStatus
+ NumericLiteralParser::GetFixedPointValue(llvm::APFloat &Result) {
+   using llvm::APFloat;
+ 
+   unsigned n = std::min(SuffixBegin - ThisTokBegin, ThisTokEnd - ThisTokBegin);
+ 
+   llvm::SmallString<16> Buffer;
+   StringRef Str(ThisTokBegin, n);
+   if (Str.find('\'') != StringRef::npos
+           || Str.find('_') != StringRef::npos  // <- SYNOPSYS
+           ) {
+     Buffer.reserve(n);
+     std::remove_copy_if(Str.begin(), Str.end(), std::back_inserter(Buffer),
+                         &isDigitSeparator);
+     Str = Buffer;
+   }
+ 
+   return Result.convertFromString(Str, APFloat::rmNearestTiesToEven);
+ }
```

## /include/clang/AST/ASTContext.h

Added the new CanQualType (see QualType) in order to handle the new fixed point type and the respective getter.
```C++
+   CanQualType SatSignedShortFractTy;
+   CanQualType SatSignedFractTy;
+   CanQualType SatSignedLongFractTy;
+   CanQualType SatSignedLongLongFractTy;
+   CanQualType SatSignedShortAccumTy;
+   CanQualType SatSignedAccumTy;
+   CanQualType SatSignedLongAccumTy;
+   CanQualType SatSignedLongLongAccumTy;
+   CanQualType Int96Ty, UnsignedInt96Ty;
```

```C++
+   QualType getInt96Ty() const { return Int96Ty; }
+   QualType getUnsignedInt96Ty() const { return UnsignedInt96Ty; }
+   QualType getSatSignedShortFractTy() const { return SatSignedShortFractTy; }
+   QualType getSatSignedFractTy() const { return SatSignedFractTy; }
+   QualType getSatSignedLongFractTy() const { return SatSignedLongFractTy; }
+   QualType getSatSignedLongLongFractTy() const { return SatSignedLongLongFractTy; }
+   QualType getSatSignedShortAccumTy() const { return SatSignedShortAccumTy; }
+   QualType getSatSignedAccumTy() const { return SatSignedAccumTy; }
+   QualType getSatSignedLongAccumTy() const { return SatSignedLongAccumTy; }
+   QualType getSatSignedLongLongAccumTy() const { return SatSignedLongLongAccumTy; }
+ 
+   QualType getSignedFractType(unsigned SizeInBits);
+   QualType getFixedPointType(const llvm::fltSemantics&) const;
+   QualType getMaximalAccumType(QualType) const;
+   QualType getAccumTypeForFractType(QualType) const;
```

As we can see here the getters return the CanQualType defined before as a QualType.

## /lib/AST/ASTContext.cpp

This file implements the AST interface.

- Specified the rank of the new type, from the smallest one (SatSignedShortFractRank) to the biggest one (SatSignedLongLongAccumRank), in the **FloatingRank** enum.

```C++
+   SatSignedShortFractRank,
+   SatSignedFractRank,
+   SatSignedLongFractRank,
+   SatSignedLongLongFractRank,
+   SatSignedShortAccumRank,
+   SatSignedAccumRank,
+   SatSignedLongAccumRank,
+   SatSignedLongLongAccumRank,
```

- Created the constructors that creates the new Type. As we can see all the correct properties are set in the method **initFixedPointType**( declared in Type.h ). For example the Fract type has 0 bits for the integer part and fbit for the rational part.

```C++
+ void ASTContext::InitBuiltinFractType(CanQualType &R, bool isSigned, unsigned fBits) {
+   BuiltinType *Ty = new (*this, TypeAlignment) BuiltinType(BuiltinType::FixedPoint);
+   Ty->initFixedPointType(isSigned, 0, fBits);
+   R = CanQualType::CreateUnsafe(QualType(Ty, 0));
+   Types.push_back(Ty);
+ }

+ void ASTContext::InitBuiltinAccumType(CanQualType &R, bool isSigned, unsigned iBits, unsigned fBits) {
+   BuiltinType *Ty = new (*this, TypeAlignment) BuiltinType(BuiltinType::FixedPoint);
+   Ty->initFixedPointType(isSigned, iBits, fBits);
+   R = CanQualType::CreateUnsafe(QualType(Ty, 0));
+   Types.push_back(Ty);
+ }

+ void ASTContext::InitBuiltinFixedPointType(CanQualType &R, BuiltinType::Kind K, unsigned GuardBits, unsigned Width) {
+   BuiltinType *Ty = new (*this, TypeAlignment) BuiltinType(K);
+   Ty->initFixedPointType(GuardBits, Width);
+   R = CanQualType::CreateUnsafe(QualType(Ty, 0));
+   Types.push_back(Ty);
+ }
```

- Initialize the fixed point type for every supported specifiers using the functions defined above

```C++
+   InitBuiltinType(Int96Ty,         BuiltinType::Int96);
+   InitBuiltinType(UnsignedInt96Ty, BuiltinType::UInt96);
+ 
+   InitBuiltinFixedPointType(SatSignedShortFractTy, BuiltinType::SatSignedShortFract, 0,	Target.getShortFractWidth());
+   InitBuiltinFixedPointType(SatSignedFractTy,	 BuiltinType::SatSignedFract, 0, Target.getFractWidth());
+   InitBuiltinFixedPointType(SatSignedLongFractTy,	BuiltinType::SatSignedLongFract, 0,	Target.getLongFractWidth());
+   InitBuiltinFixedPointType(SatSignedLongLongFractTy,	BuiltinType::SatSignedLongLongFract, 0,	Target.getLongLongFractWidth());
+   InitBuiltinFixedPointType(SatSignedShortAccumTy, BuiltinType::SatSignedShortAccum, Target.getShortAccumGuardBits(),Target.getShortAccumWidth());
+   InitBuiltinFixedPointType(SatSignedAccumTy,	BuiltinType::SatSignedAccum,Target.getAccumGuardBits(),		Target.getAccumWidth());
+   InitBuiltinFixedPointType(SatSignedLongAccumTy,	BuiltinType::SatSignedLongAccum, Target.getLongAccumGuardBits(),		Target.getLongAccumWidth());
+   InitBuiltinFixedPointType(SatSignedLongLongAccumTy,	BuiltinType::SatSignedLongLongAccum, Target.getLongLongAccumGuardBits(), Target.getLongLongAccumWidth());
```

- Implemented a function in order to calculate the correct width based on the alignement of the type

```C++
+ uint64_t FixedPointAlign(uint64_t Width, unsigned Align) {
+   if (Align > 1) {
+     assert(llvm::isPowerOf2_32(Align));
+     uint64_t t = Align - 1;
+     Width = (Width + t) & ~t;
+   }
+   return Width;
+ }
```

- Added the case that calculates the correct width for each possible new type exploiting the function declared above

```C++
//EXAMPLE : 
+     case BuiltinType::SatSignedShortFract:
+       Width = Target->getShortFractWidth();
+       Align = Target->getShortFractAlign();
+       Width = FixedPointAlign(Width,Align);
+       break;
```

- Implemented a functions that, given a size in bit return the correct QualType.

```C++
+ QualType ASTContext::getSignedFractType(unsigned SizeInBits) {
+   if (getTypeSize(SatSignedShortFractTy) == SizeInBits)
+     return SatSignedShortFractTy;
+   if (getTypeSize(SatSignedFractTy) == SizeInBits)
+     return SatSignedFractTy;
+   if (getTypeSize(SatSignedLongFractTy) == SizeInBits)
+     return SatSignedLongFractTy;
+   // TODO add long long
+   llvm_unreachable("Failed getSignedFractType");
+   return SatSignedLongFractTy;
+ }


```

- Implemented a function that returns the proper QualType that can store the result based on the precision that can handle the type.

```C++
+ QualType ASTContext::getMaximalAccumType(QualType QT) const {
+   assert(QT->isFractOrAccumType());
+   assert(QT->isSaturated());		// Don't currently support unsaturated fixed point
+   assert(QT->isSignedFixedPoint());	// Don't currently support unsigned fixed point
+   if (const Type* Ty = QT->getUnqualifiedDesugaredType()) {
+     if (const BuiltinType *BT = Ty->getAs<BuiltinType>()) {
+       switch (BT->getKind()) {
+       case BuiltinType::SatSignedShortFract:
+       case BuiltinType::SatSignedShortAccum:	return SatSignedShortAccumTy;
+       case BuiltinType::SatSignedFract:
+       case BuiltinType::SatSignedAccum:		return SatSignedAccumTy;
+       case BuiltinType::SatSignedLongFract:
+       case BuiltinType::SatSignedLongAccum:	return SatSignedLongAccumTy;
+       case BuiltinType::SatSignedLongLongFract:
+       case BuiltinType::SatSignedLongLongAccum:	return SatSignedLongLongAccumTy;
+       default: break;
+       }
+     }
+   }
+   return SatSignedLongLongAccumTy;
+ }
```

## /lib/AST/ExprConstant.cpp

The code in order to convert an int to a fixed point type has been added

```C++
+ bool FloatExprEvaluator::converttIntToFixedPoint(const Expr* E, unsigned ShiftAmount) {
+   llvm::APSInt sInt(128);
+   if (!EvaluateInteger(E, sInt, Info))
+     return false;
+   APFloat F(Result.getSemantics());
+   APFloat::opStatus S = F.convertFromAPInt(sInt, sInt.isNegative(), APFloat::rmTowardZero);
+   if (S != APFloat::opOK && S != APFloat::opInexact) // When inexact we did the best we could
+     return false;
+   llvm::APSInt shift(64);
+   shift = uint64_t(ShiftAmount);
+   if (F.Shr(shift) != APFloat::opOK)
+     return false;
+   Result = F;
+   return true;
+ }

+   case clang::ARC::BI__int_to_q7:	return converttIntToFixedPoint(E->getArg(0), 7);
+   case clang::ARC::BI__int_to_q15:	return converttIntToFixedPoint(E->getArg(0), 15);
+   case clang::ARC::BI__int_to_q31:	return converttIntToFixedPoint(E->getArg(0), 31);
+   case clang::ARC::BI__int_to_a24:	return converttIntToFixedPoint(E->getArg(0), 31);
+   case clang::ARC::BI__int_to_a40:	return converttIntToFixedPoint(E->getArg(0), 31);
+   case clang::ARC::BI__int_to_a72:	return converttIntToFixedPoint(E->getArg(0), 63);
+   case clang::ARC::BI__q7_to_int:
+   case clang::ARC::BI__q15_to_int:
+   case clang::ARC::BI__q31_to_int:
+   case clang::ARC::BI__a24_to_int:
+   case clang::ARC::BI__a40_to_int:
+   case clang::ARC::BI__a72_to_int:
+     llvm_unreachable("Fixed-to-int NYI");
+     return false;
```

For example, if we want to convert an int to a fixed point that has 0 bit for the integer part and 7 bit for the rational part we have to shift this integer 7 positions to the right.

## /lib/CodeGen/CGExprConstant.cpp

This contains code to emit Constant Expr nodes as LLVM Code. Since the Fixed Points are handled as CI ( Constant Integer ) the following code implements the proper generation of a CI given a fixed point.


```C++   
+     if (Init.isFixedPoint()) {
+       unsigned Width = DestType->getFixedPointWidth();
+       llvm::APSInt fixedPointBits(Width, /*isUnsigned=>*/ !Init.isNegative());
+       llvm::FixedPointFormat Format(DestType->isSignedFixedPoint(), DestType->getFixedPointGuardBits(), Width);
+       Init.getMachineCodeValue(fixedPointBits, llvm::APFloat::rmTowardZero, Format);
+       unsigned Size = getContext().getTypeSize(DestType.getTypePtr());
+       if (Size > Width) {
+ 	fixedPointBits = fixedPointBits.extend(Size);
+       }
+       return llvm::ConstantInt::get(VMContext, fixedPointBits);
+     } 
 ```

## /lib/CodeGen/CGExprScalar.cpp

This contains code to emit Expr nodes with scalar LLVM types as LLVM code. Here has been implemented all the functions that will emit all the operation between fixed point using custom llvm intrinsic function (we don't have the code of them).

- Extended the function **VisitFloatingLiteral** in order to return a ConstantInt (remember : the fixed point types are managed as integer type) if the destinantion type ios a fract or an accum. For example the fixed point type **fract a = 1.2** is parsed as a floating literal but its destination type is a fract then we have to generate a **ConstantInt**(1,2 * scale_factor) and not a ConstantFP.

```C++
+     QualType DestType = E->getType();
+     if (DestType->isFractOrAccumType()) {
+       llvm::APFloat Init = E->getValue();
+       unsigned Width = DestType->getFixedPointWidth();
+       llvm::APSInt fixedPointBits = llvm::ARC::getFixedPointFromFloat(Init, DestType->isSignedFixedPoint(), DestType->getFixedPointGuardBits(), Width);
+       unsigned Size = CGF.getContext().getTypeSize(DestType.getTypePtr());
+       if (Size > Width) {
+ 	fixedPointBits = fixedPointBits.extend(Size);
+       }
+       return llvm::ConstantInt::get(VMContext, fixedPointBits);
+     }
```

- Implemented all the binary operation between the fixed point type. For example the xor operation takes the two fixed point operand,converts them as integer and create a Xor operation between integer in the IR using the Builder helper object.

```C++
+   Value *EmitFractXor(const BinOpInfo &Ops) {
+     Value* Lv = convertInputToI32(Ops.LHS, getLT(Ops.E));
+     Value* Rv = convertInputToI32(Ops.RHS, getRT(Ops.E));
+     Value* Result = Builder.CreateXor(Lv, Rv, "qxor");
+     Result = truncToFract(Result, Ops.Ty);
+     return Result;
+   }
```

- Implement the functions that handle the conversion between different types, for example fract to int. As we can see also here  intrinsic functions are used.

```C++
+   Value* EmitConvertFractToInt(Value *Src, QualType SrcType, QualType DstType) {
+     llvm::Type *argTypes[] = { ConvertType(DstType), ConvertType(SrcType) };
+     llvm::Intrinsic::ID id = llvm::Intrinsic::arc_FRACT_TO_INT;
+     llvm::Function *F = CGF.CGM.getIntrinsic(id, argTypes);
+     Value* V = Builder.CreateCall(F, Src);
+     return V;
+   }
```

## /include/clang/AST/Expr.cpp

Implementation of the function **commuteFixedPoint()**,
Basically if the type of the operand involved in the expression is Fract or Accum, if we match one of the operation in the switch (only the commutative operations) then we are going to swap the SubExprs[0] and SubExprs[1] (SubExprs is an array of Stmt). This swap can be useful in order to simplify some cases when we have to deal with cast ecc.. because we know for sure that if a fixed point type is involved in the expression this will be for sure the left operand (maybe?). 

```C++
+ void BinaryOperator::commuteFixedPoint() {
+   if (getType()->isFractOrAccumType() && getLHS()->getType()->isIntegerType()) {
+     switch (getOpcode()) {
+     case BO_Mul:
+     case BO_Add:
+     case BO_EQ:
+     case BO_NE:
+     case BO_And:
+     case BO_Xor:
+     case BO_Or:
+       std::swap(SubExprs[0], SubExprs[1]);
+       break;
+     default:
+       break;
+     }
+   }
+ }
```

## /lib/Sema/SemaExpr.cpp

This file implements semantic analysis for expressions. In particular this file implements all the conversions needed in order to perform the operations between fixed point and integer / float type.

Added inside the function **handleFloatConversion()**

Basically if one of the operands is a fixed point type and the other one is an integer type the function gets the Qualtype of the fixed poit operands and cast the integer type to it.
```C++
! //<Synopsys> FixedPoint
!   if (LHSType->isFractOrAccumType() && RHSType->isIntegerType()) {
!     QualType T = S.Context.getMaximalAccumType(LHSType);
!     RHS = S.ImpCastExprToType(RHS.get(), T, CK_FloatingCast);
!     RHSType = T;
!   }
!   else if (RHSType->isFractOrAccumType() && LHSType->isIntegerType()) {
!     QualType T = S.Context.getMaximalAccumType(RHSType);
!     LHS = S.ImpCastExprToType(LHS.get(), T, CK_FloatingCast);
!     LHSType = T;
!   }
! 
!   bool LHSFloat = LHSType->isRealOrFixed();
!   bool RHSFloat = RHSType->isRealOrFixed();
```

Inside The function **UsualArithmeticConversion** is added the code in order to handle the Arithmetic conversions also for the new added type.

The function 'UsualArithmeticConversion', according to the internal documentation, performs various conversions that are common to binary operators. If both operands aren't arithmetic, this routine returns the first non-arithmetic type found.
```C++
+ #if 0
+   if (LHSType->isFractOrAccumType()) {
+     if (RHSType->isIntegerType()) {
+       RHS = UsualUnaryConversions(RHS.take());
+       return LHSType;
+     }
+   } else if (RHSType->isFractOrAccumType()) {
+     if (LHSType->isIntegerType()) {
+       LHS = UsualUnaryConversions(LHS.take());
+       return RHSType;
+     }
+   }
+ #endif
```

The **division operation** isn't supported yet and it is avoided with this snippet of code added inside **CheckMultiplyDivideOperands**. Basically if one of the operand has a fixed point type the function return an error message. This is valid even for the vector of fixed point
```C++
+   if (IsDiv && (isFixedPointOrVectorFixedPoint(LHS.get()->getType()) || isFixedPointOrVectorFixedPoint(RHS.get()->getType()))) {
+     Diag(Loc, diag::err_typecheck_invalid_operands) << LHS.get()->getType() << RHS.get()->getType();
+     return QualType();
+   }

+   if (isVectorFixedPoint(LHS.get()->getType())) {
+     const VectorType *LHSVecType = LHS.get()->getType()->getAs<VectorType>();
+     if (!eqSizedIntVector(Context, RHS.get()->getType(), LHSVecType->getElementType(), LHSVecType->getNumElements())) {
+       Diag(Loc, diag::err_typecheck_invalid_operands) << LHS.get()->getType() << RHS.get()->getType();
+       return QualType();
+     }
+     return LHS.get()->getType();
+   }
```

- **BuildFixedPointLiteral(Sema &S, NumericLiteralParser &Literal,QualType Ty, SourceLocation Loc)**

This snippet is used in order to add the FixedPointLiteral node as leaves of the AST. ( this is the exactly same thing as , for example, the IntegerLiteralNode, see schema ) 


```C++

+ static Expr *BuildFixedPointLiteral(Sema &S, NumericLiteralParser &Literal,
+ 				    QualType Ty, SourceLocation Loc) {
+ 
+   using llvm::APFloat;
+ 
+   llvm::FixedPointFormat Format(Ty->isSignedFixedPoint(), Ty->getFixedPointGuardBits(), Ty->getFixedPointWidth());
+   const llvm::fltSemantics &Sem = S.Context.getFloatTypeSemantics(Ty);
+   APFloat Val(Sem);
+ 
+   APFloat::opStatus result = Literal.GetFixedPointValue(Val);
+ 
+   // Overflow is always an error, but underflow is only an error if
+   // we underflowed to zero (APFloat reports denormals as underflow).
+   if ((result & APFloat::opOverflow) ||
+       ((result & APFloat::opUnderflow) && Val.isZero())) {
+     unsigned diagnostic;
+     SmallString<20> buffer;
+     if (result & APFloat::opOverflow) {
+       diagnostic = diag::warn_float_overflow;
+       llvm::APFloat::getFixedPointMax(Format).toString(buffer);
+     } else {
+       diagnostic = diag::warn_float_underflow;
+       llvm::APFloat::getFixedPointMin(Format).toString(buffer);
+     }
+ 
+     S.Diag(Loc, diagnostic)
+       << Ty
+       << StringRef(buffer.data(), buffer.size());
+   }
+ 
+   bool isExact = (result == APFloat::opOK);
+   return FloatingLiteral::Create(S.Context, Val, isExact, Ty, Loc);
+ }
```

- **isFixedPointOrVectorFixedPoint(QualType Ty)**

Checks if the Type is a Fixed one. If the element is a vector, get its type and call this function recursively on the retrieved type.

```C++
+ static bool isFixedPointOrVectorFixedPoint(QualType Ty) {
+   if (Ty->isFractOrAccumType()) return true;
+   if (Ty->isVectorType())
+     return isFixedPointOrVectorFixedPoint(Ty->getAs<VectorType>()->getElementType());
+   return false;
+ }
```

- ** isVectorFixedPoint(QualType Ty)**

The same as the function above, but it checks only if its a vector and not only a fixed point value

```C++
+ static bool isVectorFixedPoint(QualType Ty) {
+   if (Ty->isVectorType())
+     return isFixedPointOrVectorFixedPoint(Ty->getAs<VectorType>()->getElementType());
+   return false;
+ }
```

## /lib/Sema/SemaOverload.cpp

Added the handling of tags in order to deal with the cast to and from fixed point type. 

For example :

```C++
case ICK_Floating_Integral:
   FloatingIntegralConversion:
!  if (FromType->isRealFloatingType() && ToType->isIntegralType(Ctx)) {
    return NK_Type_Narrowing;
```

The fixed point type is identified by isRealFloatingType() function.

## /lib/Sema/SemaType.cpp

The function **ConvertDeclSpecToType** returns a QualType given a declaration specifier. It has been added this code in order to handling all the cases for the new added types included the error messages (like 'unsigned fixed point is not supported')
```C++
+   case DeclSpec::TST_fract:
+     if (!Context.getTargetInfo().hasFixedPoint()) {
+       S.Diag(DS.getTypeSpecTypeLoc(), diag::err_fixed_point_unsupported);
+       Result = Context.IntTy;
+       declarator.setInvalidType(true);
+       break;
+     }
+     if (DS.getTypeSpecSign() == DeclSpec::TSS_unsigned) {
+       S.Diag(DS.getTypeSpecTypeLoc(), diag::err_unsigned_fixed_point_unsupported);
+       Result = Context.IntTy;
+       declarator.setInvalidType(true);
+       break;
+     }
+     if (!DS.isSaturated()) {
+       S.Diag(DS.getTypeSpecTypeLoc(), diag::err_unsat_fixed_point_unsupported);
+       Result = Context.IntTy;
+       declarator.setInvalidType(true);
+       break;
+     }
+     switch (DS.getTypeSpecWidth()) {
+     case DeclSpec::TSW_unspecified:
+       Result = Context.SatSignedFractTy;
+       break;
+     case DeclSpec::TSW_short:
+       Result = Context.SatSignedShortFractTy;
+       break;
+     case DeclSpec::TSW_long:
+       Result = Context.SatSignedLongFractTy;
+       break;
+     case DeclSpec::TSW_longlong:
+       Result = Context.SatSignedLongLongFractTy;
+       break;
+     }
+     break;
+   case DeclSpec::TST_accum:
+     if (!Context.getTargetInfo().hasFixedPoint()) {
+       S.Diag(DS.getTypeSpecTypeLoc(), diag::err_fixed_point_unsupported);
+       Result = Context.IntTy;
+       declarator.setInvalidType(true);
+       break;
+     }
+     if (DS.getTypeSpecSign() == DeclSpec::TSS_unsigned) {
+       S.Diag(DS.getTypeSpecTypeLoc(), diag::err_unsigned_fixed_point_unsupported);
+       Result = Context.IntTy;
+       declarator.setInvalidType(true);
+       break;
+     }
+     if (!DS.isSaturated()) {
+       S.Diag(DS.getTypeSpecTypeLoc(), diag::err_unsat_fixed_point_unsupported);
+       Result = Context.IntTy;
+       declarator.setInvalidType(true);
+       break;
+     }
+     switch (DS.getTypeSpecWidth()) {
+     case DeclSpec::TSW_unspecified:
+       Result = Context.SatSignedAccumTy;
+       break;
+     case DeclSpec::TSW_short:
+       Result = Context.SatSignedShortAccumTy;
+       break;
+     case DeclSpec::TSW_long:
+       Result = Context.SatSignedLongAccumTy;
+       break;
+     case DeclSpec::TSW_longlong:
+       Result = Context.SatSignedLongLongAccumTy;
+       break;
+     }
+     break;
+   case DeclSpec::TST_int96: // __int96 is used just to represent a 72-bit accumulator as an integer object
+     if (!Context.getTargetInfo().hasInt96Type())
+       S.Diag(DS.getTypeSpecTypeLoc(), diag::err_int96_unsupported);
+     if (DS.getTypeSpecSign() == DeclSpec::TSS_unsigned)
+       Result = Context.UnsignedInt96Ty;
+     else
+       Result = Context.Int96Ty;
+     break;
```

## /lib/Serialization/ASTCommon.cpp - ASTReader.cpp

ASTCommon.cpp file defines common functions that both ASTReader and ASTWriter use, while ASTReader.cpp reads AST files.
In these file this code in order to retrieve the proper ID for the new types that has been added.

- **ASTCommon.cpp**

```C++
+   case BuiltinType::UInt96:			ID = PREDEF_TYPE_UInt96_ID;			break;
+   case BuiltinType::Int96:			ID = PREDEF_TYPE_Int96_ID;			break;
+   case BuiltinType::SatSignedShortFract:	ID = PREDEF_TYPE_SatSignedShortFract_ID;	break;
+   case BuiltinType::SatSignedFract:		ID = PREDEF_TYPE_SatSignedFract_ID;		break;
+   case BuiltinType::SatSignedLongFract:		ID = PREDEF_TYPE_SatSignedLongFract_ID;		break;
+   case BuiltinType::SatSignedLongLongFract:	ID = PREDEF_TYPE_SatSignedLongLongFract_ID;	break;
+   case BuiltinType::SatSignedShortAccum:	ID = PREDEF_TYPE_SatSignedShortAccum_ID;	break;
+   case BuiltinType::SatSignedAccum:		ID = PREDEF_TYPE_SatSignedAccum_ID;		break;
+   case BuiltinType::SatSignedLongAccum:		ID = PREDEF_TYPE_SatSignedLongAccum_ID;		break;
+   case BuiltinType::SatSignedLongLongAccum:	ID = PREDEF_TYPE_SatSignedLongLongAccum_ID;	break;
```

- **ASTReader.cpp**

```C++
+   case PREDEF_TYPE_UInt96_ID:			T = Context.getUnsignedInt96Ty(); break;
+   case PREDEF_TYPE_Int96_ID:			T = Context.getInt96Ty(); break;
+   case PREDEF_TYPE_SatSignedShortFract_ID:	T = Context.getSatSignedShortFractTy(); break;
+   case PREDEF_TYPE_SatSignedFract_ID:		T = Context.getSatSignedFractTy(); break;
+   case PREDEF_TYPE_SatSignedLongFract_ID:	T = Context.getSatSignedLongFractTy(); break;
+   case PREDEF_TYPE_SatSignedLongLongFract_ID:	T = Context.getSatSignedLongLongFractTy(); break;
+   case PREDEF_TYPE_SatSignedShortAccum_ID:	T = Context.getSatSignedShortAccumTy(); break;
+   case PREDEF_TYPE_SatSignedAccum_ID:		T = Context.getSatSignedAccumTy(); break;
+   case PREDEF_TYPE_SatSignedLongAccum_ID:	T = Context.getSatSignedLongAccumTy(); break;
+   case PREDEF_TYPE_SatSignedLongLongAccum_ID:	T = Context.getSatSignedLongLongAccumTy(); break;
```

# Questions

- In the file Expr.cpp what does it mean " Allow an intrinsic function call to be used for a fixed point static initializer"? what is the point of this code?

```C++
+   case CallExprClass:{
+     const CallExpr* call = cast<CallExpr>(this);
+     if (unsigned id = call->getBuiltinCallee()) {
+       if (call->getNumArgs() == 1) {
+ 	switch (id) {
+ 	case clang::ARC::BI__int_to_q7:
+ 	case clang::ARC::BI__int_to_q15:
+ 	case clang::ARC::BI__int_to_q31:
+ 	case clang::ARC::BI__int_to_a24:
+ 	case clang::ARC::BI__int_to_a40:
+ 	case clang::ARC::BI__int_to_a72:
+ 	case clang::ARC::BI__q7_to_int:
+ 	case clang::ARC::BI__q15_to_int:
+ 	case clang::ARC::BI__q31_to_int:
+ 	case clang::ARC::BI__a24_to_int:
+ 	case clang::ARC::BI__a40_to_int:
+ 	case clang::ARC::BI__a72_to_int:
+ 	  return call->getArg(0)->isConstantInitializer(Ctx, false, Culprit);
+ 	default: break;
+ 	}
+       }
+     }
+   } break;
```
